<a href="https://colab.research.google.com/github/srinidhi183/CODE---SRINIDHI-/blob/StatsandML/100001318__Exercise__4_Srinidhi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

url = "https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv"
df = pd.read_csv(url)

# Display the first few rows
print(df.head())


   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500   NaN        S  




Handling missing values


Standardization (Z-score) and Normalization (Min-Max Scaling)


In [6]:
missing_values = df.isnull().sum() # Change 'df' to 'data'
print(missing_values)

# Impute Age with the median
df['Age'].fillna(df['Age'].median(), inplace=True)

# Impute Embarked with the mode
df['Embarked'].fillna(df['Embarked'].mode()[0], inplace=True)


from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df[['Age', 'Fare']] = scaler.fit_transform(df[['Age', 'Fare']])


from sklearn.preprocessing import MinMaxScaler
min_max_scaler = MinMaxScaler()
df[['Age', 'Fare']] = min_max_scaler.fit_transform(df[['Age', 'Fare']])

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64


<ipython-input-6-c85399d84bdc>:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Age'].fillna(df['Age'].median(), inplace=True)
<ipython-input-6-c85399d84bdc>:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try usin


Handling Noise
Injecting Random Noise and Removing It

In [7]:
import numpy as np
noise = np.random.normal(0, 1, size=df['Age'].shape)
df['Age'] += noise

df['Age'] = df['Age'].rolling(window=3, min_periods=1).mean()


df['Age'] = df['Age'].rolling(window=3, min_periods=1).mean()


Handling Outliers
Detecting Outliers Using Z-score

In [8]:
from scipy.stats import zscore
df['Age_zscore'] = zscore(df['Age'])
outliers = df[df['Age_zscore'].abs() > 3]
print(outliers)


df = df[df['Age_zscore'].abs() <= 3]


df['Age'] = np.log1p(df['Age'])

     PassengerId  Survived  Pclass                             Name     Sex  \
238          239         0       2  Pengelly, Mr. Frederick William    male   
409          410         0       3               Lefebre, Miss. Ida  female   

          Age  SibSp  Parch Ticket      Fare Cabin Embarked  Age_zscore  
238  1.799734      0      0  28665  0.020495   NaN        S    3.049242  
409  1.788852      3      1   4133  0.049708   NaN        S    3.026842  


<ipython-input-8-1c9935857310>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Age'] = np.log1p(df['Age'])



Feature Selection

Correlation Matrix

In [9]:


# Select only numeric columns before calculating correlation
numeric_df = df.select_dtypes(include=['number'])

# Calculate the correlation matrix
correlation_matrix = numeric_df.corr()

# Print the correlation with 'Survived'
print(correlation_matrix['Survived'].sort_values(ascending=False))

Survived       1.000000
Fare           0.257008
Parch          0.081985
Age           -0.003267
PassengerId   -0.005852
SibSp         -0.033873
Age_zscore    -0.035298
Pclass        -0.338468
Name: Survived, dtype: float64




Mutual Information


In [12]:
from sklearn.feature_selection import mutual_info_classif
from sklearn.preprocessing import LabelEncoder

# Create a LabelEncoder object
le = LabelEncoder()

# Select object (string) columns for encoding, EXCLUDING 'Name'
object_cols = df.select_dtypes(include=['object']).columns
object_cols = object_cols[object_cols != 'Name']  # Exclude 'Name' column

# Apply Label Encoding to each object column
for col in object_cols:
    df[col] = le.fit_transform(df[col])

# Calculate mutual information
mi = mutual_info_classif(df.drop('Survived', axis=1), df['Survived'])
print(mi)

[0.01367488 0.07434569 0.00143307 0.15018552 0.03403334 0.04928009
 0.0103305  0.11604709 0.13344917 0.03844652 0.05210608 0.03498229]




Wrapper Methods
Recursive Feature Elimination
Backward Elimination

In [11]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder

# Create a LabelEncoder object
le = LabelEncoder()

# Select only object (string) columns for encoding
object_cols = df.select_dtypes(include=['object']).columns

# Apply Label Encoding to each object column
for col in object_cols:
    df[col] = le.fit_transform(df[col])

# Create the model and RFE object
model = LogisticRegression()
rfe = RFE(estimator=model, n_features_to_select=5)

# Fit the RFE object to the transformed data
fit = rfe.fit(df.drop('Survived', axis=1), df['Survived'])

# Print the results
print(fit.support_)
print(fit.ranking_)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

[False  True False  True False  True False False  True False  True False]
[6 1 7 1 2 1 4 8 1 5 1 3]



Lasso Regression

In [13]:













from sklearn.linear_model import Lasso
lasso = Lasso(alpha=0.01)
lasso.fit(df.drop('Survived', axis=1), df['Survived'])
print(lasso.coef_)



[-4.95623930e-07 -1.05158821e-01 -6.13969442e-05 -4.76434218e-01
  0.00000000e+00 -1.90088849e-02 -0.00000000e+00 -1.44376644e-04
  0.00000000e+00 -7.68112805e-04 -2.86584916e-02 -0.00000000e+00]
